In [32]:
import fiona
from shapely.geometry import shape, mapping
from shapely.ops import transform
from shapely.geometry import Polygon
import copy

# origShapeFile = 'E:/OSM/Schutzgebiete/Schongebiete-Alt/Schongebiete-4326.shp' # epsg:4326
origShapeFile = 'E:/OSM/Schutzgebiete/Schongebiete.geojson' # epsg:4326
newShapeFile = 'E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten-epsg4326.shp' # epsg:4326

shapesUpdateFile = 'E:/OSM/Schutzgebiete/New/new-shapes-dav.shp'
shapesIdenticalFile = 'E:/OSM/Schutzgebiete/New/same-shapes-dav.shp'
shapesDeletedFile = 'E:/OSM/Schutzgebiete/New/deleted-shapes-dav.shp'
shapesDeletedGeoJson = 'E:/OSM/Schutzgebiete/deleted-shapes-dav.geojson'

In [27]:
def read_geojson(geojsonFile):
    oldFeatures = []
    with fiona.open(geojsonFile) as input:
        schema = input.schema
        for feat in input:
            if 'classification' in feat['properties'] and feat['properties']['classification'] != '':
                if feat['geometry'] != None and len(feat['geometry']['coordinates'][0]) > 2:
                    oldFeatures.append(feat)
    size = len(oldFeatures)
    print(f"OSM-Gebiete in {geojsonFile}: {size}")
    return oldFeatures, schema

# read and 3D to 2D convert DAV shapefile
def read_dav(shapeFile):
    newFeatures = []
    with fiona.open(shapeFile) as input:
        schema = input.schema
        # crs = input.crs
        # driver = input.driver
        for feat in input:
            if 'Kategorie' in feat['properties'] and feat['properties']['Kategorie'] != None and feat['properties']['Kategorie'] == "FFH-Gebiet (A)":
                print("Ignoring FFH-Gebiet (A)")
                continue
            if feat['geometry'] != None:
                if len(feat['geometry']['coordinates']) > 1:
                    # multipolygons - code to be improved...
                    for pfeat in feat['geometry']['coordinates']:
                        try: # some are len(1) lists, some aren't ?!
                            poly = Polygon(pfeat[0])
                        except:
                            poly = Polygon(pfeat)
                        poly = transform(lambda x, y, z=None: (x, y), poly)
                        feat2 = copy.deepcopy(feat)
                        feat2['geometry'] = mapping(poly)
                        newFeatures.append(feat2)
                    continue
                if len(feat['geometry']['coordinates'][0]) < 3:
                    print("Skipping 2-point line")
                    continue
                # transform 3D to 2D
                poly = shape(feat['geometry'])
                poly = transform(lambda x, y, z=None: (x, y), poly)
                feat['geometry'] = mapping(poly)
                newFeatures.append(feat)

    size = len(newFeatures);          
    print(f"Gebiete in {shapeFile}: {size}")
    return newFeatures, schema

In [28]:
oldFeatures, schemaGJ = read_geojson(origShapeFile)
newFeatures, schemaShp = read_dav(newShapeFile)

OSM-Gebiete in E:/OSM/Schutzgebiete/Schongebiete.geojson: 430
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Gebiete in E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten-epsg4326.shp: 410


In [29]:
# iterate old shapes and check if they still exist in new shapes
# -> writes shapesDeletedFile
delCount = 0 
foundCount = 0
missingFeaturesOut = []
sameFeatures = []

for oldFeature in oldFeatures:
    oldGeom = Polygon(shape(oldFeature['geometry']).exterior)
    oldGeomB = oldGeom.buffer(0.0001)
    found = False
    for newFeature in newFeatures:
        try:
            newGeom = Polygon(shape(newFeature['geometry']))
            newGeomB = newGeom.buffer(0.0001)
        except Exception as ex:
            continue
        try:
            iou = oldGeomB.intersection(newGeomB).area / oldGeomB.union(newGeomB).area
        except Exception as ex:
            print(ex)
            print(newGeom)
            print(oldGeom)
        if iou > 0.995:
            found = True
            foundCount +=1
            sameFeatures.append(oldFeature)
    if not found:
        delCount += 1
        missingFeaturesOut.append(oldFeature)
        # print(f"Not found: {oldFeature['properties']}")

print(f"Gefundene Gebiete = {foundCount}")
print(f"Gelöschte Gebiete = {delCount} -> in {shapesDeletedFile}")

Gefundene Gebiete = 344
Gelöschte Gebiete = 86 -> in E:/OSM/Schutzgebiete/New/deleted-shapes-dav.shp


In [30]:
with fiona.open(shapesDeletedFile, 'w', crs={'init':'epsg:4326'}, driver='ESRI Shapefile', schema=schemaGJ) as out:
    for f in missingFeaturesOut:
        out.write(f)

In [33]:
with fiona.open(shapesDeletedGeoJson, 'w', crs={'init':'epsg:4326'}, driver='GeoJSON', schema=schemaGJ) as out:
    for f in missingFeaturesOut:
        out.write(f)